# House price prediction from Airbnb housing and socio-demographic indexes from New York City, 2017

* Brocco Mattia **|** 873058
* Piccolo Giulio **|** 874727

In [2]:
import dtale
import geopy
import statistics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
%matplotlib inline

import Data_cleaner as cle
import Cleaner_2 as res
%reload_ext autoreload
%autoreload 2

cleaner = cle.cleaners()
shaper = res.reshaper()

In [ ]:
### URL datasets
# https://www.kaggle.com/new-york-city/nypd-motor-vehicle-collisions
# https://www.kaggle.com/new-york-city/nyc-property-sales#nyc-rolling-sales.csv
# http://insideairbnb.com/get-the-data.html

### Giulio: "/Users/giuliopiccolo/Desktop/lab/project/dataset/"
### Mattia: ""/Users/matti/Desktop/BA Second Year/3. Lab of Information systems and analytics/3. Project/"

In [3]:
path = "/Users/matti/Desktop/BA Second Year/3. Lab of Information systems and analytics/3. Project/"
airbnb = pd.read_csv(path + "NYC_Airbnb_2017.csv", low_memory = False)
house_price = pd.read_csv(path + "NYC_house_price.csv")
crashes = pd.read_csv(path + "NYC_collision.csv", low_memory = False)

In [4]:
neighborhoods = pd.read_excel(path + "NYC_neighborhoods.xlsx")
neigh_list = [tuple(x) for x in neighborhoods.values.tolist()]

## Data cleaning of NYC House Price dataset

To assess the proportion to cut off the mean to compute the trimmed mean, we performed this analysis and the graphic result shows how a good approximation can be made at the cutoff of:
* 0.011 for "land_square_feet"
* 0.08 for "gross_square_feet"

```python
# This study was performed during data cleaning operations of the abovementioned dataframe
x_var = np.arange(0, 0.5, 0.001)
y_var = np.array( [round(scipy.stats.trim_mean(df_price['land_square_feet'], x), 3) for x in x_var] )
z_soglia = [1650 for x in range(len(x_var))] # visual check for interception with y_var

plt.figure(figsize=(15,6))
plt.plot(y_var, linewidth = 5, color = "#A4063E")
plt.plot(z_soglia, linewidth = 2, color = "blue")

def find_nearest(array, value):
    """
    * idx is the position (index) of the value we are looking for,
    using the function armin to find the minimum value
    * then return the actual value found
    """
    idx = (np.abs(array - value)).argmin()
    return array[idx]

y_var_list = list(y_var)
x_var_list = list(x_var)

# Visualize the threshold for the truncated mean
print(  x_var_list[y_var_list.index(find_nearest(y_var, z_soglia[0]))]  )
```

## Data cleasing of NYC house prices dataset

In [5]:
df_price = cleaner.house_price_cleaner(house_price)
df_price.shape

(69607, 20)

In [ ]:
# Heatmap of correlations
plt.figure(figsize = (10,8))
corr = df_price.corr()
sns.heatmap(corr, annot = True)

#### Code to possibly slice dataset according to price thresholds
#df_sliced = df_price["sale_price"].copy()
#df_sliced_list = [x for x in df_sliced.tolist() if 100000 < x < 10000000]
#sns.distplot(df_sliced_list)

In [ ]:
# Barplot of available values
gess = df_price.copy()
count = []
for var in gess.columns:
    length = gess[var].count()
    count.append(length)
    
count_pct = np.round(100 * pd.Series(count) / len(gess), 2)
print(count_pct.tolist())

plt.figure(figsize=(10,6), dpi = 300)
plt.barh(gess.columns, count_pct)
plt.title('Count of available data in percent', fontsize=15)
plt.show()

In [ ]:
# TO ADD
# interative data visualization using plotly package (https://plot.ly)

## Data cleasing of NYC car accidents dataset

In [6]:
df_crash = cleaner.crash_cleaner(crashes, neigh_list)
df_crash.shape

(204908, 25)

### How many missing values do we have?

The following heatmap serves as a way to visualize the extent to which a column contains empty cells.
- Crashes that involve more than 2 vehicles are higly populated of missing values as expected, since most of the time crashes involves two vehicles only
- The 'borough' column needs to be improved, and we did it by gathering the coordinates of the accident to identify the borough name

In [ ]:
crash_to_viz = df_crash.copy()
plt.figure(figsize=(10,7))
sns.heatmap(crash_to_viz.isnull(), cbar = False, cmap = 'viridis')

In [ ]:
from math import cos, asin, sqrt
# Haversine formula
def distance_viz(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # math.PI / 180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a)) #2 * R; R = 6371 km

# get the closest point 
def closest_viz(data, v):
    return min(data, key = lambda p: distance_viz(v[0], v[1], p[0], p[1]) )

borough_dict = {"BRONX" : [40.837048, -73.865433], "BROOKLYN" : [40.650002, -73.949997],
                "MANHATTAN" : [40.758896, -73.985130], "QUEENS" : [40.742054,-73.769417],
                "STATEN ISLAND" : [40.579021,-74.151535]}

boroughs = [(*value, key) for key, value in borough_dict.items()]

crash_to_viz['borough'] = crash_to_viz.apply( lambda row:
    closest_viz(boroughs, [row['latitude'], row['longitude']])[2]
    if row['borough'] is None else row['borough'], axis=1)

In [ ]:
# The same heatmap is performed to check the quantity of NaN values after the abovementioned map has been applied
plt.figure(figsize=(10,7))
sns.heatmap(crash_to_viz.isnull(), cbar = False, cmap = 'viridis')

## Data cleasing of Airbnbs of NYC dataset
The "airbnb" dataframe required only a little effort in cleasing operations. As shown through the chunck below, which output is $2.741 \% $, this dataset is characterized by a very low rate of empty cells.
```python
print("Missing values: ",  df_blocks.isnull().sum().sum() , " | Perc of total: ",
      round(100*df_blocks.isnull().sum().sum() / (df_blocks.shape[0]*df_blocks.shape[1]), 3))
```
So, we decided to all drop the rows that presented NaN values, since they represent only a small share of the total.

```python
total = df1.isnull().sum().sort_values(ascending = False)
percent = (df1.isnull().sum()/df1.isnull().count()).sort_values(ascending = False)

missing_data = pd.concat([total, percent], axis = 1, keys = ['NAs', 'Percent']) # df with columns, nan values and %
plt.plot( missing_data['Percent'] )
missing_data.drop(missing_data[missing_data.Percent < 0.2].index, inplace = True)
```
After a further evaluation of the columns, we assessed that only ```"reviews_per_month"``` and ```"last_review"``` had more than more than 20% of the whole column filled with NaN values. So, we dropped those two columns and used ```.dropna()``` to delete other few rows with any NaN values.

In [7]:
df_airbnb = cleaner.airbnb_cleaner(airbnb, neigh_list)
df_airbnb.shape

True


(48486, 14)

## NYC Neighborhoods coordinates and neigborhoods applied to all dataframes

The following code was used to create the dataset
```python
# !!!!!!!! DA SISTEMARE PER CREDIBILITA' (FOR CON LISTE E OUTPUT E' UN DATAFRAME)
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="project_lisa")
location = geolocator.geocode("West Harlem")
print((location.latitude, location.longitude))
```

## Merge of all datasets

Il dataset che dobbiamo ottenere ha come riga una casa, le colonne riguardanti gli incidenti e gli airbnb sono mergate rispetto al quartiere di appartenenza della casa.
- ogni dataset va trasformato (tipo pivot) in modo che abbia come riga un quartiere e come colonna degli "aggregate" per ogni quartiere
- poi si fa un merge degli altri due addosso a df_price (che di suo avrà già le coordinate latitudine e longitudine)

ES: Airbnb
- drop delle colonne "id","name","host_id","host_name","borough"
- COLONNE: Neighborhood, Moda di romm_type, media di price, media di min_price, media di reviews, media di listings, media di availability /// RIGHE: tante quante sono i quartieri

ES: Crash
- drop delle colonne "accident_date", "accident_time", "borough", "location", tutte quelle con 1-2-3-4-5
- COLONNE: Neighborhood, media di persons injured, media di persons killed, media di number_of_pedestrians_injured, media di number_of_pedestrians_killed, media di number_of_cyclist_injured, media di number_of_cyclist_killed, media di number_of_motorist_injured, media di number_of_motorist_killed

Tutte le abitazione nello stesso quartiere avranno le stesse informazioni rispetto agli airbnb.

In [9]:
crash_pivot = shaper.PVT_crash(df_crash)
crash_pivot

,neighborhood,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed
0,AIRPORT LA GUARDIA,0.003236,0.000000,0.249191,0.000000,0.000000,0.006472,0.252427,0.006472
1,ALPHABET CITY,0.023984,0.000076,0.208753,0.000840,0.034143,0.000687,0.267206,0.001604
2,ANNADALE,0.000000,0.000000,0.273333,0.000000,0.006667,0.000000,0.280000,0.000000
3,ARDEN HEIGHTS,0.000000,0.000000,0.220472,0.000000,0.015748,0.000000,0.236220,0.000000
4,ARROCHAR,0.004695,0.000000,0.173709,0.000000,0.009390,0.000000,0.187793,0.000000
...,...,...,...,...,...,...,...,...,...
230,WOODHAVEN,0.020571,0.000000,0.195429,0.000000,0.053714,0.000000,0.266286,0.000000
231,WOODLAWN,0.009985,0.000000,0.290323,0.000768,0.062980,0.000000,0.364823,0.000000
232,WOODROW,0.045455,0.000000,0.227273,0.000000,0.045455,0.000000,0.318182,0.000000
233,WOODSIDE,0.040084,0.000000,0.199895,0.000527,0.046941,0.000000,0.286920,0.000527


In [11]:
bnb_pivot = shaper.PVT_bnb(df_airbnb)
bnb_pivot

,neighborhood,availability_365,calculated_host_listings_count,minimum_nights,number_of_reviews,price,room_type
0,AIRPORT LA GUARDIA,217.500000,3.666667,1.000000,122.333333,53.416667,Private room
1,ALPHABET CITY,102.004677,1.261927,3.434986,20.738073,168.082320,Entire home/apt
2,ARROCHAR,128.666667,1.000000,3.333333,19.666667,91.333333,Entire home/apt
3,ARROCHAR-SHORE ACRES,204.071429,1.214286,3.000000,17.785714,90.857143,Private room
4,ARVERNE,280.461538,6.692308,4.730769,20.346154,109.000000,Private room
...,...,...,...,...,...,...,...
213,WOODHAVEN,247.108108,1.594595,3.081081,39.081081,72.162162,Private room
214,WOODLAWN,248.093750,1.562500,1.750000,24.406250,91.906250,Private room
215,WOODROW,262.000000,1.000000,2.000000,7.000000,225.000000,Private room
216,WOODSIDE,152.736944,2.210832,4.187621,17.237911,86.709865,Private room


In [15]:
merge_fun = shaper.merger(df_price, bnb_pivot)
merge_finale = shaper.merger(merge_fun, crash_pivot)
merge_finale

,borough,neighborhood,building_class_category,tax_class_at_present,block,lot,building_class_at_present,address,apartment_number,zip_code,...,price,room_type,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,C2,153 AVENUE B,,10009,...,168.08232,Entire home/apt,0.023984,0.000076,0.208753,0.00084,0.034143,0.000687,0.267206,0.001604
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,C4,154 EAST 7TH STREET,,10009,...,168.08232,Entire home/apt,0.023984,0.000076,0.208753,0.00084,0.034143,0.000687,0.267206,0.001604
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,C2,301 EAST 10TH STREET,,10009,...,168.08232,Entire home/apt,0.023984,0.000076,0.208753,0.00084,0.034143,0.000687,0.267206,0.001604
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,406,32,C4,210 AVENUE B,,10009,...,168.08232,Entire home/apt,0.023984,0.000076,0.208753,0.00084,0.034143,0.000687,0.267206,0.001604
4,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,387,153,D9,629 EAST 5TH STREET,,10009,...,168.08232,Entire home/apt,0.023984,0.000076,0.208753,0.00084,0.034143,0.000687,0.267206,0.001604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69602,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,B9,37 QUAIL LANE,,10309,...,225.00000,Private room,0.045455,0.000000,0.227273,0.00000,0.045455,0.000000,0.318182,0.000000
69603,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,B9,32 PHEASANT LANE,,10309,...,225.00000,Private room,0.045455,0.000000,0.227273,0.00000,0.045455,0.000000,0.318182,0.000000
69604,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,B2,49 PITNEY AVENUE,,10309,...,225.00000,Private room,0.045455,0.000000,0.227273,0.00000,0.045455,0.000000,0.318182,0.000000
69605,5,WOODROW,22 STORE BUILDINGS,4,7100,28,K6,2730 ARTHUR KILL ROAD,,10309,...,225.00000,Private room,0.045455,0.000000,0.227273,0.00000,0.045455,0.000000,0.318182,0.000000


In [ ]:
copia_fin = merge_finale.copy()
copia_fin = copia_fin.dropna()
copia_fin.shape

In [ ]:
copia_fin

## First try of ML Prediction

Steps to follow:
1. Descriptive analysis on the Data – 50% time
2. Data treatment (Missing value and outlier fixing) – 40% time
3. Data Modelling – 4% time
4. Estimation of performance – 6% time

In [ ]:
mylist2 = df_airbnb['neighbourhood'].tolist()
mylist2 = list(dict.fromkeys(mylist2))
mylist2